In [5]:
from utils import *
import os
import cv2
import numpy as np
import random as ra

In [6]:
def include_border(x1,y1,x2,y2, border):
    x1+=border
    x2+=border
    y1+=border
    y2+=border
    return x1,y1,x2,y2

def correct_rect_ratio(x1,y1,x2,y2):
    w,h=(abs(x1-x2), abs(y1-y2))
    center=((x1+x2)/2,(y1+y2)/2)
    h=w/5 #ratio 1:5
    y1=int(center[1]-h/2)
    y2=int(center[1]+h/2)
    return x1,y1,x2,y2

def add_margin(x1,y1,x2,y2):
    w,h=(abs(x1-x2), abs(y1-y2))
    center=((x1+x2)/2,(y1+y2)/2)
    w_margin=0.2*w/2
    h_margin=0.2*h/2
    x1-=w_margin
    x2+=w_margin
    y1-=h_margin
    y2+=h_margin
    return int(x1),int(y1),int(x2),int(y2)

def add_border(image):
    border=IMG_SAFETY_BORDER
    return cv2.copyMakeBorder(image, border, border, border, border, cv2.BORDER_REFLECT)

def update_optimal_dimensions(x1,y1,x2,y2):
    w,h=(abs(x1-x2), abs(y1-y2))
    global w_max,h_max,w_min,h_min
    w_max=max(w,w_max)
    h_max=max(h,h_max)
    w_min=min(w,w_min)
    h_min=min(h,h_min) 
    
def save_extracted_image(img, scene_name, filename, subpath, index=None, png=False):
    postfix="_"+str(index) if index else ""
    filename=filename.replace(".jpg", postfix+".jpg")
    if png:
        filename=filename.replace(".jpg", ".png")
    path = DS_path+subpath
    os.chdir(path) 
    try:
        os.mkdir(scene_name)
    except FileExistsError:
        None
    os.chdir(scene_name) 
    cv2.imwrite(filename, img)
    
def imread_resized(scene_name, filename, label_resolution):
    filepath=get_filepath(scene_name, filename)
    image = cv2.imread(filepath)
    return cv2.resize(image, label_resolution)
    
def read_channels(scene_name, filename):
    filepath=CHANNELS_PATH+scene_name+"/"+filename
    ch14_path=filepath.replace(".jpg", "_ch14"+".png")
    ch58_path=filepath.replace(".jpg", "_ch58"+".png")
    ch14 = cv2.imread(ch14_path, cv2.IMREAD_UNCHANGED)
    ch58 = cv2.imread(ch58_path, cv2.IMREAD_UNCHANGED)
    return [ch14,ch58]
    
def extract_pallet_rectangles_from_image(regions, border, image_bordered, filename, scene_name, channels_mode=False):
    for j in range(len(regions)):
        (x1,y1),(x2,y2)=regions[j]
        x1,y1,x2,y2=include_border(x1,y1,x2,y2,border)
        x1,y1,x2,y2=correct_rect_ratio(x1,y1,x2,y2)
        update_optimal_dimensions(x1,y1,x2,y2)
        x1,y1,x2,y2=add_margin(x1,y1,x2,y2)
        if channels_mode:
            cropped_ms = [ch[y1:y2, x1:x2] for ch in image_bordered]
            cropped_ms = [cv2.resize(ch, (120,24)) for ch in cropped_ms]
            save_extracted_image(cropped_ms[0], scene_name, filename, "processed_scenes/pallet_rectangles_channels",str(j)+"_ch14",True)
            save_extracted_image(cropped_ms[1], scene_name, filename, "processed_scenes/pallet_rectangles_channels",str(j)+"_ch58",True)
            global pallet_rectangles_channels
            pallet_rectangles_channels.append(cropped_ms)
        else: 
            cropped_m = image_bordered[y1:y2, x1:x2] #fixme calculaing max may be required
            cropped_m = cv2.resize(cropped_m, (120,24))
    #         save_extracted_image(cropped_m, scene_name, filename, "processed_scenes/pallet_rectangles_margin", j)
            global pallet_rectangles
            pallet_rectangles.append(cropped_m)
        
def convert_to_channels(image, filename, scene_name):
    img = cv2.UMat(image)
    img = cv2.GaussianBlur(img, (3,3), sigmaX=0, sigmaY=0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
    gradX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3, scale=1 )
    gradY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3, scale=1 )
    mag, ori = cv2.cartToPolar(gradX, gradY, angleInDegrees=True)

    ori_ranges = [(0,60), (60,120), (120,180), (180,240), (240,300), (300,360)]
    oris=[cv2.inRange(ori,l,r) for (l,r) in ori_ranges]
    mags= [cv2.copyTo(mag, ori_x) for ori_x in oris]
    channels=[gray,mag]+mags
    channels=[cv2.UMat.get(ch) for ch in channels]
    channels=np.moveaxis(channels, 0, -1)
    ch14,ch58=np.array_split(channels, 2,2)
    save_extracted_image(ch14, scene_name, filename, "processed_scenes/channels","ch14",True)
    save_extracted_image(ch58, scene_name, filename, "processed_scenes/channels","ch58",True)

def extract_background_rectangles_from_image(regions, image, filename, scene_name, channels_mode=False):
    if regions:
        pallets_y1=99999
        pallets_y2=0
        sample_pallet_w=0
        y1s=[]
        y2s=[]
        hs=[]
        for j in range(len(regions)):
            (x1,y1),(x2,y2)=regions[j]
            x1,y1,x2,y2=correct_rect_ratio(x1,y1,x2,y2)
            x1,y1,x2,y2=add_margin(x1,y1,x2,y2)
            w,h=(abs(x1-x2), abs(y1-y2))
            y1s.append(y1)
            y2s.append(y2)
            h=abs(y1-y2)
            hs.append(h)
        h=int(np.mean(hs))
        w=5*h
        y1=min(y1s)
        y2=max(y2s)
        upper_background = image[0:y1]
        lower_background = image[y2:]
        
        img_h,img_w,_=image[0].shape if channels_mode else image.shape
        sampling_x1=0
        sampling_x2=img_w-w
        sampling_upper_y1=0
        sampling_upper_y2=y1-h
        sampling_lower_y1=y2
        sampling_lower_y2=img_h-h
        
        can_sample_upper=sampling_upper_y2>0
        can_sample_lower=sampling_lower_y2>sampling_lower_y1
        
        samples_count=4
        if (can_sample_upper or can_sample_lower) and sampling_x2>0: 
            for i in range(samples_count):
                sampled_x=ra.randint(sampling_x1, sampling_x2)
                sampling_done=False
                if i%2 and can_sample_upper:
                    sampled_y=ra.randint(sampling_upper_y1, sampling_upper_y2)
                    sampling_done=True
                elif can_sample_lower:
                    sampled_y=ra.randint(sampling_lower_y1, sampling_lower_y2)
                    sampling_done=True
                if sampling_done:
                    if channels_mode:
                        samples=[ch[sampled_y:sampled_y+h, sampled_x:sampled_x+w] for ch in image]
                        samples=[cv2.resize(sample, (120,24)) for sample in samples]
                        save_extracted_image(samples[0], scene_name, filename, "processed_scenes/backgrounds_channels",str(i)+"_ch14",True)
                        save_extracted_image(samples[1], scene_name, filename, "processed_scenes/backgrounds_channels",str(i)+"_ch58",True)
                        global background_rectangles_channels
                        background_rectangles_channels.append(samples)
                    else:
                        sample=image[sampled_y:sampled_y+h, sampled_x:sampled_x+w]
                        sample=cv2.resize(sample, (120,24))
    #                     save_extracted_image(sample, scene_name, filename, "processed_scenes/backgrounds", i)
                        global background_rectangles
                        background_rectangles.append(sample)

### Higher level functions

In [7]:
def process_image_channels_extraction(filename, scene_name, label_resolution, rects):
    image=imread_resized(scene_name, filename, label_resolution)
    convert_to_channels(image, filename, scene_name)
    
def process_image_structural_extraction(filename, scene_name, label_resolution, rects):
    channels=read_channels(scene_name, filename)
    channels_bordered=[add_border(ch) for ch in channels]
    extract_pallet_rectangles_from_image(rects, channels_bordered, filename, scene_name, True)
    extract_background_rectangles_from_image(rects, channels, filename, scene_name, True)
    
def process_image_rgb_extraction(filename, scene_name, label_resolution, rects):
    image=imread_resized(scene_name, filename, label_resolution)
    image_bordered = add_border(image)
    extract_pallet_rectangles_from_image(rects, image_bordered, filename, scene_name)
    extract_background_rectangles_from_image(rects, image, filename, scene_name)

def process_image(filename, scene_name, label_resolution, rects, fun):
    fun(filename, scene_name, label_resolution, rects)

def process_scene(scene_name, fun):
    include,_,_,_,label_resolution=read_info(scene_name)
    if include:
        filenames = get_image_names(scene_name)
        labels_dict=read_labels_dict(scene_name)
        for i in range(len(filenames)):
            filename=filenames[i]
            rects=labels_dict[filename]
            process_image(filename, scene_name, label_resolution, rects, fun)

def process_data(fun):
    scene_names = get_scene_names()
    for i in range(len(scene_names)):
        scene_name=scene_names[i]
        process_scene(scene_name, fun)

### Main

In [ ]:
w_max=h_max=0
w_min=h_min=99999

pallet_rectangles_rgb=[]
background_rectangles_rgb=[]
background_rectangles_channels=[]
pallet_rectangles_channels=[]

MODE_CHANNELS_EXTRACTION=process_image_channels_extraction
MODE_STRUCTURAL_EXTRACTION=process_image_structural_extraction
MODE_RGB_EXTRACTION=process_image_rgb_extraction
MODE=MODE_CHANNELS_EXTRACTION

process_data(MODE)

In [ ]:
print("w_max: ",w_max,"h_max: ",h_max)
print("w_min: ",w_min,"h_min: ",h_min)

In [ ]:
os.chdir(PROJECT_PATH) 

if MODE==MODE_STRUCTURAL_EXTRACTION:
    np.save("background_rectangles_channels",background_rectangles_channels)
    np.save("pallet_rectangles_channels",pallet_rectangles_channels)
    print(np.array(background_rectangles_channels).shape)
    print(np.array(pallet_rectangles_channels).shape)
elif MODE==MODE_STRUCTURAL_EXTRACTION:
    np.save("background_rectangles_rgb",background_rectangles_rgb)
    np.save("pallet_rectangles_rgb",pallet_rectangles_rgb)
    print(np.array(background_rectangles_channels).shape)
    print(np.array(pallet_rectangles_channels).shape)